In [ ]:
!pip install pymupdf faiss-cpu sentence-transformers transformers accelerate einops bitsandbytes
!pip install unsloth
!pip install flask flask-cors
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
from huggingface_hub import login
login("token")

In [ ]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from gtts import gTTS
import os, socket, threading
import fitz
import faiss
import numpy as np
import re
import uuid
import torch
from sentence_transformers import SentenceTransformer
from unsloth import FastLanguageModel
from transformers import logging

logging.set_verbosity_error()

# === Constants ===
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
LLAMA_MODEL = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
CHUNK_SIZE = 250
CHUNK_OVERLAP = 50
MAX_TOKENS = 512
MIN_SIMILARITY_THRESHOLD = 0.60
PORT = 5080
UPLOAD_FOLDER = "uploads"

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# === Text Processing ===
def extract_all_pages(pdf_path):
    doc = fitz.open(pdf_path)
    return [page.get_text() for page in doc]

def split_text_into_chunks(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

def embed_chunks(chunks, model_name=EMBEDDING_MODEL):
    embedder = SentenceTransformer(model_name)
    embeddings = embedder.encode(chunks, show_progress_bar=True)
    return np.array(embeddings), embedder

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    return index

def load_llama_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=LLAMA_MODEL,
        max_seq_length=4096,
        load_in_4bit=True
    )
    model = FastLanguageModel.get_peft_model(model)
    return model, tokenizer

def ask_llama(model, tokenizer, question, retrieved_chunks):
    context = "\n\n".join(retrieved_chunks)
    prompt = f"""أنت مساعد ذكي ومتخصص في فهم النصوص العربية. مهمتك هي الإجابة على السؤال التالي اعتمادًا فقط على المعلومات المذكورة في السياق أدناه.

لا تضف أي معلومات من خارج النص.
استخدم اللغة العربية الفصحى.
اجعل الإجابة مختصرة وواضحة، ولا تتجاوز ثلاث جمل.

السياق:
{context}

السؤال: {question}

الإجابة:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.5,
        top_p=0.85,
        do_sample=True,
        repetition_penalty=1.15,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded.split("الإجابة:")[-1].strip()
    answer = re.sub(r'[a-zA-Z]+', '', answer).strip()
    answer = re.sub(r'[^\u0600-\u06FF0-9\s،؟.:؛]+', '', answer)
    sentences = re.split(r'[.؟:؛]', answer)
    return "،".join(sentences[:3]).strip()

def arabic_pdf_qa(pdf_path, question, last_page=None):
    all_text = extract_all_pages(pdf_path)
    visible_text = "".join(all_text if last_page is None else all_text[:last_page])
    chunks = split_text_into_chunks(visible_text)
    if not chunks:
        return "⚠️ لا يوجد محتوى كافٍ للإجابة.", None

    embeddings, embedder = embed_chunks(chunks)
    index = build_faiss_index(embeddings)
    question_embedding = embedder.encode([question])
    faiss.normalize_L2(question_embedding)
    similarities, top_indices = index.search(np.array(question_embedding), 5)

    relevant_chunks = [chunks[i] for i, score in zip(top_indices[0], similarities[0]) if score >= MIN_SIMILARITY_THRESHOLD]
    if not relevant_chunks:
        return "❌  لا يمكنني الإجابة على هذا السؤال لأن محتواه يقع خارج نطاق الصفحات المحددة", None

    model, tokenizer = load_llama_model()
    answer = ask_llama(model, tokenizer, question, relevant_chunks)
    return answer, relevant_chunks

@app.route("/", methods=["POST"])
def ask():
    try:
        pdf = request.files["pdf"]
        question = request.form["question"]
        last_page = int(request.form.get("last_page", 0)) or None

        filepath = os.path.join(UPLOAD_FOLDER, pdf.filename)
        pdf.save(filepath)

        answer, _ = arabic_pdf_qa(filepath, question, last_page)

        #Generate a unique filename using UUID
        unique_id = str(uuid.uuid4())
        speech_filename = f"{unique_id}_speech.mp3"
        speech_path = os.path.join(UPLOAD_FOLDER, speech_filename)

        #Convert answer to speech
        speech = gTTS(answer, lang='ar')
        speech.save(speech_path)

        return jsonify({
            "answer": answer,
            "audioUrl": f"/uploads/{speech_filename}"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_from_directory(UPLOAD_FOLDER, filename)

def is_port_available(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        return sock.connect_ex(("0.0.0.0", port)) != 0

def run_flask():
    if is_port_available(PORT):
        app.run(host="0.0.0.0", port=PORT)
    else:
        print(f"Port {PORT} is already in use.")

threading.Thread(target=run_flask).start()

<ipython-input-3-36e7b6369264>:12: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5080
 * Running on http://172.28.0.12:5080


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!./cloudflared tunnel --url http://localhost:5080

2025-04-26T03:18:00Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-04-26T03:18:00Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-04-26T03:18:05Z INF +--------------------------------------------------------------------------------------------+
2025-04-26T03:18:05Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-04-26T03:18:05Z INF |  https://rolls-copyright-relations-tub.trycloudflare.c

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 03:28:11] "POST / HTTP/1.1" 200 -


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 03:29:37] "POST / HTTP/1.1" 200 -


2025-04-26T03:30:30Z INF Initiating graceful shutdown due to signal interrupt ...
2025-04-26T03:30:31Z ERR Failed to serve tunnel connection error="context canceled" connIndex=0 event=0 ip=198.41.200.43
2025-04-26T03:30:31Z INF Retrying connection in up to 1s connIndex=0 event=0 ip=198.41.200.43
2025-04-26T03:30:31Z ERR no more connections active and exiting
2025-04-26T03:30:31Z INF Tunnel server stopped
2025-04-26T03:30:31Z INF Metrics server stopped


In [ ]:
pdf_path = "/content/test.pdf"
question = "ما تداعيات المجاعة الكبرى على لبنان خلال الحرب العالمية الأولى؟"

answer = arabic_pdf_qa(pdf_path, question, last_page=None)

print("\n🧠 الجواب:\n", answer)

📄 Extracting content...
🔀 Chunking...
🔎 Embedding chunks...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]


🧠 الجواب:
 كانت المجاعة الكبرى التي تعرض لها لبنان خلال الحرب العالمية الأولى، نتيجة للحصار الذي فرضه العثمانيون وتقلّصا إمدادات الغذائية، أدت إلى موت مئات من الأطفال والأشخاص البالغون، وقد كان ذلك تأثيرًا كبيرًا على  والثقافة في جبل لبنان، حيث انعكس هذا الوقوع على نمط الحياة اليومية والاقتصادية والثقافية، وبسبب هذه الظروف الصعبة، دعت الحكومة الفرنسية إلى إنشاء دولة لبنان الكبيرة في عام 1920، والتي ضمت جميع المناطق في البلاد


In [ ]:
pdf_path = "/content/test.pdf"
last_page =  17
question = "ما تداعيات المجاعة الكبرى على لبنان خلال الحرب العالمية الأولى؟"
answer = arabic_pdf_qa(pdf_path, question, last_page=None)
print("\n🧠 الجواب:\n", answer)

📄 Extracting content...
🔀 Chunking...
🔎 Embedding chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Searching for relevant content...
🦙 Generating response with LLaMA...
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

🧠 الجواب:
 كانت المجاعة الكبرى التي تعرض لها لبنان خلال الحرب العالمية الأولى، نتيجة للحصار الذي فرضه العثمانيون وتقلّصا إمدادات الغذائية، أدت إلى موت مئات من الأطفال والأشخاص البالغون، وقد كان ذلك تأثيرًا كبيرًا على  والثقافة في جبل لبنان، حيث انعكس هذا الوقوع على نمط الحياة اليومية والاقتصادية والثقافية، وبسبب هذه الظروف الصعبة، دعت الحكومة الفرنسية إلى إنشاء دولة لبنان الكبيرة في عام 1920، والتي ضمت جميع المناطق في البلاد


In [ ]:
pdf_path = "/content/test.pdf"
last_page =  3
question = "ما تداعيات المجاعة الكبرى على لبنان خلال الحرب العالمية الأولى؟"
answer = arabic_pdf_qa(pdf_path,  question, last_page)
print("\n🧠 الجواب:\n", answer)

📄 Extracting content...
🔀 Chunking...
🔎 Embedding chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Searching for relevant content...

🧠 الجواب:
 ❌ لا يمكنني الإجابة على هذا السؤال لأن محتواه يقع خارج نطاق الصفحات المحددة.
